In [1]:
"""
CSTR (Continuously Stirred Tank Reactor) Environment Demo
========================================================

This demo showcases the CSTR environment from pc-gym, which simulates
a continuously stirred tank reactor for chemical process control.

The CSTR is a well-established model that's thoroughly tested and stable,
making it perfect for learning and experimentation.

State Variables (3 total):
- Ca: Concentration of reactant A (mol/L)
- T: Temperature (K)
- Cb: Concentration of reactant B (mol/L)

Action Variables (1 total):
- Tc: Coolant temperature (K)

Observations (3 total):
- Ca, T, Cb: Concentration A, Temperature, Concentration B
"""

import numpy as np
from pcgym import make_env
from rl.cstr.optimization.visualization import (
    plot_state_variables,
    plot_control_actions,
    plot_reward_evolution,
    plot_summary_dashboard)
from rl.cstr.optimization.load_config_files import load_and_create_env_params

In [10]:
# ============================================================================
# ENVIRONMENT SETUP
# ============================================================================

# Load configuration from YAML file
config_path = "/workspace/general_projects/rl_cstr_optimization/config/environments/cstr_environment.yaml"
env_params = load_and_create_env_params(config_path)

# Extract action space bounds from env_params for use in the notebook
a_space = env_params['a_space']
o_space = env_params['o_space']

print(f"Configuration loaded: {env_params}")
print(f"Action space: {a_space}")
print(f"Observation space: {o_space}")


Configuration loaded: {'N': 10, 'tsim': 26, 'SP': {'Ca': [0.85, 0.85, 0.85, 0.9, 0.9, 0.9, 0.87, 0.87, 0.87, 0.87]}, 'o_space': {'low': array([  0.7, 300. ,   0.8]), 'high': array([  1. , 350. ,   0.9])}, 'a_space': {'low': array([295]), 'high': array([302])}, 'x0': array([  0.8, 330. ,   0.8]), 'r_scale': {'ca': 1000}, 'model': 'cstr', 'normalise_a': True, 'normalise_o': True, 'noise': True, 'integration_method': 'casadi', 'noise_percentage': 0.001}
Action space: {'low': array([295]), 'high': array([302])}
Observation space: {'low': array([  0.7, 300. ,   0.8]), 'high': array([  1. , 350. ,   0.9])}


In [3]:
# # ============================================================================
# # ENVIRONMENT SETUP
# # ============================================================================

# # Define simulation parameters for CSTR reactor
# T = 26  # Simulation time (hours)
# nsteps = 10  # Number of steps for our demo

# # Define action space bounds for CSTR reactor
# # Action space: [Tc] - Coolant temperature (K)
# # Based on the example from pc-gym documentation
# a_space = {
#     'low': np.array([295]),  # Lower bound for coolant temperature (K)
#     'high': np.array([302])  # Upper bound for coolant temperature (K)
# }

# # Define observation space bounds for CSTR reactor
# # Observations: [Ca, T, Cb] - Concentration A, Temperature, Concentration B
# # Based on the example from pc-gym documentation
# o_space = {
#     'low': np.array([0.7, 300, 0.8]),   # Lower bounds: [Ca_min, T_min, Cb_min]
#     'high': np.array([1.0, 350, 0.9])   # Upper bounds: [Ca_max, T_max, Cb_max]
# }

# # Define initial conditions for CSTR reactor
# # Initial state: [Ca, T, Cb] - Concentration A, Temperature, Concentration B
# # Based on the example from pc-gym documentation
# x0 = np.array([0.8, 330, 0.8])  # Initial conditions: [Ca_0, T_0, Cb_0]

# # Define set points for the CSTR reactor
# # Based on the example from pc-gym documentation
# # We'll create a simple set point profile for Ca (concentration A)
# # Make sure we have exactly nsteps elements
# if nsteps >= 3:
#     # Create a profile similar to the original example
#     third = nsteps // 3
#     SP = {
#         'Ca': [0.85 for i in range(third)] + 
#                 [0.9 for i in range(third)] + 
#                 [0.87 for i in range(nsteps - 2*third)],  # Target concentration A (mol/L)
#     }
# else:
#     # For small nsteps, use constant set point
#     SP = {
#         'Ca': [0.85 for i in range(nsteps)],  # Target concentration A (mol/L) - constant set point
#     }

# # Define reward scaling for CSTR reactor
# # Based on the example from pc-gym documentation
# r_scale = {'Ca': 1e3}  # Reward scaling for concentration A

# # Create environment parameters dictionary
# # Based on the example from pc-gym documentation
# env_params = {
#     'N': nsteps,                    # Number of steps
#     'tsim': T,                      # Simulation time
#     'SP': SP,                       # Set points for control objectives
#     'o_space': o_space,             # Observation space bounds (3D)
#     'a_space': a_space,             # Action space bounds (1D)
#     'x0': x0,                      # Initial conditions (3D)
#     'r_scale': r_scale,            # Reward scaling
#     'model': 'cstr',                # Model type - CSTR is well-tested
#     'normalise_a': True,            # Normalize actions
#     'normalise_o': True,            # Normalize observations
#     'noise': True,                  # Enable noise for realism
#     'integration_method': 'casadi', # Use CasADi integration (stable for CSTR)
#     'noise_percentage': 0.001,      # Noise percentage
# }

In [11]:
# Create the CSTR environment with proper parameters
# The environment simulates a continuously stirred tank reactor for chemical process control
env = make_env(env_params)

# Reset the environment to get initial state
# This returns the initial observation (concentrations at reactor outlet)
initial_observation, initial_info = env.reset()

print("=" * 60)
print("CSTR REACTOR DEMO - 10 STEP SIMULATION")
print("=" * 60)
print(f"Initial observation: {initial_observation}")
print(f"Observation shape: {initial_observation.shape}")
print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")
print("=" * 60)

CSTR REACTOR DEMO - 10 STEP SIMULATION
Initial observation: [-0.33333328  0.2        -1.00000024]
Observation shape: (3,)
Action space: Box(-1.0, 1.0, (1,), float32)
Observation space: Box(-1.0, 1.0, (3,), float32)


/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:236: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/workspace/general_projects/rl_cstr_optimization/.venv/lib/python3.11/site-packages/gymnasium/spaces/box.py:306: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


In [12]:
# ============================================================================
# DATA STORAGE FOR ANALYSIS
# ============================================================================

# Lists to store data for visualization and analysis
observations = [initial_observation]  # Store all observations (concentrations at outlet)
actions = []       # Store all actions taken
rewards = []       # Store all rewards received
states = []        # Store full state information if available

In [13]:
# ============================================================================
# MAIN SIMULATION LOOP - 10 STEPS
# ============================================================================

print("\nStarting 10-step simulation...")
print("-" * 40)

for step in range(10):
    print(f"\nStep {step + 1}/10:")
    
    # ========================================================================
    # ACTION SELECTION
    # ========================================================================
    
    # For this demo, we'll use a simple strategy:
    # - Moderate coolant temperature control
    # - Stay within the safe operating range
    
    # Since actions are normalized, we need to provide values between 0 and 1
    # These will be automatically scaled to the actual bounds defined in env_params
    action = np.array([
        0.5,    # Tc: Coolant temperature (normalized) - moderate value
    ])
    
    # Alternative: Random action within normalized bounds
    # action = np.random.uniform(0, 1, 1)
    
    # Calculate denormalized action values for display
    # Denormalize: actual_value = low + (normalized_value * (high - low))
    denorm_action = np.array([
        a_space['low'][0] + action[0] * (a_space['high'][0] - a_space['low'][0]),  # Tc
    ])
    
    print(f"  Normalized action: {action}")
    print(f"  Denormalized action: {denorm_action}")
    print(f"  - Coolant temperature: {denorm_action[0]:.2f} K")
    
    # ========================================================================
    # ENVIRONMENT STEP
    # ========================================================================
    
    # Execute the action in the environment
    # This advances the simulation by one time step
    # Returns: new_observation, reward, terminated, truncated, info
    observation, reward, terminated, truncated, info = env.step(action)
    
    print(f"  New observation: {observation}")
    print(f"  Reward: {reward:.4f}")
    print(f"  Terminated: {terminated}")
    print(f"  Truncated: {truncated}")
    
    # ========================================================================
    # DATA STORAGE
    # ========================================================================
    
    # Store the data for later analysis
    observations.append(observation.copy())
    actions.append(action.copy())
    rewards.append(reward)
    
    # ========================================================================
    # TERMINATION CHECK
    # ========================================================================
    
    # Check if the episode has ended
    if terminated or truncated:
        print(f"  Episode ended at step {step + 1}")
        break



Starting 10-step simulation...
----------------------------------------

Step 1/10:
  Normalized action: [0.5]
  Denormalized action: [298.5]
  - Coolant temperature: 298.50 K
  New observation: [ 1.12700046e-01 -1.30195354e-02  1.19209332e-07]
  Reward: -0.0003
  Terminated: False
  Truncated: False

Step 2/10:
  Normalized action: [0.5]
  Denormalized action: [298.5]
  - Coolant temperature: 298.50 K
  New observation: [ 1.63531495e-01 -1.17155302e-02  1.19209332e-07]
  Reward: -0.0006
  Terminated: False
  Truncated: False

Step 3/10:
  Normalized action: [0.5]
  Denormalized action: [298.5]
  - Coolant temperature: 298.50 K
  New observation: [1.56853147e-01 9.23848935e-03 1.19209332e-07]
  Reward: -0.0007
  Terminated: False
  Truncated: False

Step 4/10:
  Normalized action: [0.5]
  Denormalized action: [298.5]
  - Coolant temperature: 298.50 K
  New observation: [ 0.15871919 -0.01702228  1.00000048]
  Reward: -0.0007
  Terminated: False
  Truncated: False

Step 5/10:
  Normaliz

In [14]:
# ============================================================================
# SIMULATION COMPLETE - ANALYSIS AND VISUALIZATION
# ============================================================================

print("\n" + "=" * 60)
print("SIMULATION COMPLETE - ANALYSIS")
print("=" * 60)

# Convert lists to numpy arrays for easier analysis
observations = np.array(observations)
actions = np.array(actions)
rewards = np.array(rewards)

print(f"Total steps completed: {len(observations)}")
print(f"Average reward: {np.mean(rewards):.4f}")
print(f"Total reward: {np.sum(rewards):.4f}")


SIMULATION COMPLETE - ANALYSIS
Total steps completed: 10
Average reward: -0.0003
Total reward: -0.0030


In [15]:
# ============================================================================
# VISUALIZATION
# ============================================================================

In [16]:
# Create all four plots
plot_state_variables(observations)

In [17]:
# Create all four plots
plot_control_actions(actions, a_space)


In [18]:
plot_reward_evolution(rewards)

In [19]:
plot_summary_dashboard(observations, actions, rewards, a_space)

In [ ]:
# Close the environment to free resources
env.close()